# OR Advent 2024 December 1: meeting rooms (Kotlin)

## Assignment

Hi, I'm ÉléonORe, and I'm struggling to organize a series of events...

Each event requires a dedicated room.

Some events have overlapping participants, so I can't schedule them in the same room at the same time.

How many rooms do I need? How can I assign each event to a room so that no two overlapping events are scheduled in the same one?

Ideally, I want to minimize the total number of rooms used.

Can you help me solve this problem?

## Dependencies

This Kotlin Jupyter Notebook solves a school timetabling problem with [Timefold Solver](https://timefold.ai/open-source-solver), the open source planning solver AI. We add it as a dependency:

In [1]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.16.0")


## Domain

We need to assign events to timeslots and rooms:

In [2]:
import java.time.LocalDateTime

data class Timeslot(
    val start: LocalDateTime
)

In [3]:
data class Room(
    val id: String
)

This class changes during planning. Timefold Solver fills in the `@PlanningVariable ` annotated fields for all instances:

In [4]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import ai.timefold.solver.core.api.domain.variable.PlanningVariable

@PlanningEntity
data class Event(
    @PlanningId val id: String) {
    
    val conflictingEvents: Set<Event> = mutableSetOf<Event>()

    @PlanningVariable
    var timeslot: Timeslot? = null
    @PlanningVariable
    var room: Room? = null

    // No-arg constructor required for Timefold
    constructor() : this("")
}

## Constraints

The solver takes into account hard and soft constraints:

In [5]:
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.Joiners
import java.time.Duration

class OrAdventConstraintProvider : ConstraintProvider {

    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint>? {
        return arrayOf(
            // Hard constraints
            roomConflict(constraintFactory),
            // Soft constraints
        )
    }

    fun roomConflict(constraintFactory: ConstraintFactory): Constraint {
        // A room can accommodate at most one event at the same time.
        return constraintFactory
            // Select each pair of 2 different lessons ...
            .forEachUniquePair(
                Event::class.java,
                // ... in the same timeslot ...
                Joiners.equal(Event::timeslot),
                // ... in the same room ...
                Joiners.equal(Event::room)
            )
            // ... and penalize each pair with a hard weight.
            .penalize(HardSoftLongScore.ONE_HARD)
            .asConstraint("or-advent", "Room conflict");
    }

}

### The dataset class

This class represents the dataset to solve:

In [6]:
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore

@PlanningSolution
data class OrAdventDataset(
    
    @ValueRangeProvider
    val timeslots: List<Timeslot>,
    @ValueRangeProvider
    val rooms: List<Room>,
    @PlanningEntityCollectionProperty
    val events: List<Event>) {

    @PlanningScore
    var score: HardSoftLongScore? = null

    // No-arg constructor required for Timefold
    constructor() : this(emptyList(), emptyList(), emptyList())

}

## Input reader

In [7]:
import java.io.File

fun readDataset(): OrAdventDataset {
    val timeslots = mutableListOf<Timeslot>()
    var nextDateTime = LocalDateTime.of(2024,12,25,9,0)
    for (i in 1..16) {
        timeslots.add(Timeslot(nextDateTime))
        nextDateTime = nextDateTime.plusMinutes(30)
    }
    val rooms = mutableListOf<Room>()
    for (i in 1..30) {
        rooms.add(Room("$i"))
    }

    val input: String = File("./instance.txt").readText()
    val lines = input.lines().filter {
        it.isNotBlank() && !it.startsWith("#") // Skip empty or commented lines
    }
    val (numberOfEvents, _) = lines.first().split(" ").map { it.toInt() }
    val events = mutableListOf<Event>()
    for (i in 1..numberOfEvents) {
        events.add(Event("$i"))
    }
    val eventsMap = events.map { it.id to it }.toMap()
    
    lines.drop(1).forEach { line ->
        val (_, eventId1, eventId2) = line.split(" ")
        val event1 = eventsMap[eventId1]
        val event2 = eventsMap[eventId2]
        
        // event1!!.conflictingEvents.add(event2)
        // event2!!.conflictingEvents.add(event1)
        println("$eventId1 --- $eventId2")
    }
    return OrAdventDataset(timeslots, rooms, events)
}

## Solve it

Configure and run the solver:

In [8]:
import ai.timefold.solver.core.config.solver.SolverConfig
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver

val solverFactory: SolverFactory<OrAdventDataset> = SolverFactory.create(SolverConfig()
        .withSolutionClass(OrAdventDataset::class.java)
        .withEntityClasses(Event::class.java)
        .withConstraintProviderClass(OrAdventConstraintProvider::class.java)
        // The solver runs only for 5 seconds on this small dataset.
        // It's recommended to run for at least 5 minutes ("5m") otherwise.
        .withTerminationSpentLimit(Duration.ofSeconds(5)))

val problem: OrAdventDataset = readDataset()

println("Solving the problem ...")
val solver: Solver<OrAdventDataset> = solverFactory.buildSolver()
val solution: OrAdventDataset = solver.solve(problem)
println("Solving finished with score (${solution.score}).")

1 --- 5
1 --- 7
1 --- 9
1 --- 10
1 --- 11
1 --- 12
1 --- 13
1 --- 14
1 --- 15
1 --- 16
1 --- 18
1 --- 19
1 --- 22
1 --- 25
1 --- 27
1 --- 28
1 --- 30
1 --- 31
1 --- 32
1 --- 33
1 --- 35
1 --- 36
1 --- 37
1 --- 39
1 --- 40
1 --- 41
1 --- 42
1 --- 45
1 --- 46
1 --- 47
1 --- 51
1 --- 53
1 --- 54
1 --- 55
1 --- 58
1 --- 59
1 --- 60
1 --- 62
1 --- 63
1 --- 65
1 --- 69
1 --- 70
1 --- 71
1 --- 72
1 --- 75
1 --- 81
1 --- 84
1 --- 87
1 --- 88
1 --- 89
1 --- 91
1 --- 93
1 --- 95
1 --- 96
1 --- 98
2 --- 3
2 --- 5
2 --- 6
2 --- 7
2 --- 10
2 --- 12
2 --- 13
2 --- 16
2 --- 17
2 --- 19
2 --- 20
2 --- 21
2 --- 22
2 --- 23
2 --- 25
2 --- 26
2 --- 27
2 --- 28
2 --- 30
2 --- 31
2 --- 32
2 --- 33
2 --- 34
2 --- 37
2 --- 40
2 --- 41
2 --- 43
2 --- 44
2 --- 45
2 --- 46
2 --- 47
2 --- 48
2 --- 49
2 --- 50
2 --- 51
2 --- 52
2 --- 54
2 --- 56
2 --- 57
2 --- 58
2 --- 59
2 --- 62
2 --- 63
2 --- 66
2 --- 70
2 --- 81
2 --- 82
2 --- 83
2 --- 84
2 --- 85
2 --- 86
2 --- 87
2 --- 88
2 --- 96
2 --- 100
3 --- 4
3 --- 5


## Visualize the solution

In [9]:
val eventByRoomMap = solution.events.groupBy { Pair(it.timeslot, it.room) }
HTML(buildString {
    append("<p style='font-size: x-large'>Score: ${solution.score}</p>")
    append("<p><b>By room</b></p>")
    append("<table><tr><th/>")
    for (room in solution.rooms) {
        append("<th>${room.id}</th>")
    }
    append("</tr>")
    for (timeslot in solution.timeslots) {
        append("<tr><th>${timeslot.start}</th>")
        for (room in solution.rooms) {
            val cellEvents = eventByRoomMap.get(Pair(timeslot, room)) ?: emptyList()
            append("<td>")
            append(cellEvents.map { it.id }.joinToString(", "))
            append("</td>")
        }
        append("</tr>")
    }
    append("</table>")
})

Score: 0hard/0soft By room 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 2024-12-25T09:00 1 17 33 49 65 81 97 2024-12-25T09:30 2 18 34 50 66 82 98 2024-12-25T10:00 3 19 35 51 67 83 99 2024-12-25T10:30 4 20 36 52 68 84 100 2024-12-25T11:00 5 21 37 53 69 85 2024-12-25T11:30 6 22 38 54 70 86 2024-12-25T12:00 7 23 39 55 71 87 2024-12-25T12:30 8 24 40 56 72 88 2024-12-25T13:00 9 25 41 57 73 89 2024-12-25T13:30 10 26 42 58 74 90 2024-12-25T14:00 11 27 43 59 75 91 2024-12-25T14:30 12 28 44 60 76 92 2024-12-25T15:00 13 29 45 61 77 93 2024-12-25T15:30 14 30 46 62 78 94 2024-12-25T16:00 15 31 47 63 79 95 2024-12-25T16:30 16 32 48 64 80 96

## Statistics

For a big dataset, a schedule visualization is often too verbose.
Let's visualize the solution through statistics:

In [10]:
%use kandy

In [11]:
val roomIds = solution.rooms.map { it.id }
val eventCounts = solution.rooms.map { room -> solution.events.filter{ it.room == room }.count() }

plot {
    layout.title = "Events per room"
    bars {
        x(roomIds) { axis.name = "Rooms" }
        y(eventCounts) { axis.name = "Events" }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="oyG9G4"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Events per room"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":"Rooms"
},{
"aesthetic":"y",
"name":"Events",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30"],
"y":[16.0,16.0,16.0,16.0,16.0,16.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
},
"sampling":"none",
"position":"dodge",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("oyG9G4");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 
 
 7 
 
 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 
 
 9 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 11 
 
 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 
 
 13 
 
 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 
 
 17 
 
 
 
 
 
 
 
 
 18 
 
 
 
 
 
 
 
 
 19 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 21 
 
 
 
 
 
 
 
 
 22 
 
 
 
 
 
 
 
 
 23 
 
 
 
 
 
 
 
 
 24 
 
 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 
 
 26 
 
 
 
 
 
 
 
 
 27 
 
 
 
 
 
 
 
 
 28 
 
 
 
 
 
 
 
 
 29 
 
 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Events per room 
 
 
 
 
 Events 
 
 
 
 
 Rooms

## Analyze the score

Let's break down the score per constraint:

In [12]:
import ai.timefold.solver.core.api.solver.SolutionManager

val solutionManager = SolutionManager.create(solverFactory)
val scoreAnalysis = solutionManager.analyze(solution)

And visualize it:

In [13]:
HTML(buildString {
    append("<p style='font-size: x-large'>Score: ${scoreAnalysis.score}</p>")
    append("<ul>")
    for (constraint in scoreAnalysis.constraintMap().values) {
        append("<li>${constraint.constraintRef().constraintName}: ${constraint.score.toShortString()}</li>")
    }
    append("</ul>")
})

Score: 0hard/0soft Room conflict: 0

## Conclusion

To learn more about planning optimization, visit [timefold.ai](https://timefold.ai).